# Доклад: Структурные паттерны — Компоновщик, Мост, Легковес

**Структурные паттерны** отвечают за построение удобных и расширяемых иерархий классов и объектов. Сегодня мы рассмотрим три мощных паттерна, которые решают разные задачи, связанные с композицией объектов:

*   **Компоновщик (Composite):** Позволяет работать с древовидной структурой объектов так, как если бы это был единичный объект.
*   **Мост (Bridge):** Разделяет большую иерархию классов на две независимые, позволяя им развиваться по-отдельности.
*   **Легковес (Flyweight):** Экономит память, позволяя разделять общее состояние между множеством объектов.

Давайте разберем каждый из них на примерах с исполняемым кодом.

## 1. Паттерн "Компоновщик" (Composite)
**Суть:** *Позволяет клиентам единообразно обращаться как к отдельным объектам, так и к их группам.*

**Аналогия:** Папка в файловой системе. Вы можете узнать размер как одного файла (простой объект), так и всей папки (составной объект, размер которого — сумма размеров всего содержимого). Операция одна и та же.

### Пример на Python

In [ ]:
from abc import ABC, abstractmethod
from typing import List

class Graphic(ABC):
    @abstractmethod
    def draw(self):
        pass

class Circle(Graphic):
    def __init__(self, name):
        self.name = name

    def draw(self):
        print(f"  Рисуем круг '{self.name}'")

class Dot(Graphic):
    def __init__(self, name):
        self.name = name

    def draw(self):
        print(f"  Рисуем точку '{self.name}'")

class CompoundGraphic(Graphic):
    def __init__(self, name):
        self.name = name
        self._children: List[Graphic] = []

    def add(self, child: Graphic):
        self._children.append(child)

    def remove(self, child: Graphic):
        self._children.remove(child)

    def draw(self):
        print(f"--- Рисуем составную фигуру '{self.name}' ---")
        for child in self._children:
            child.draw()
        print(f"--- Конец фигуры '{self.name}' ---")

if __name__ == "__main__":
    all_graphics = CompoundGraphic("Вся страница")
    all_graphics.add(Dot("Точка 1"))
    all_graphics.add(Circle("Круг 1"))

    group = CompoundGraphic("Группа фигур")
    group.add(Circle("Внутренний круг"))
    group.add(Dot("Внутренняя точка"))

    all_graphics.add(group)

    all_graphics.draw()

--- Рисуем составную фигуру 'Вся страница' ---
  Рисуем точку 'Точка 1'
  Рисуем круг 'Круг 1'
--- Рисуем составную фигуру 'Группа фигур' ---
  Рисуем круг 'Внутренний круг'
  Рисуем точку 'Внутренняя точка'
--- Конец фигуры 'Группа фигур' ---
--- Конец фигуры 'Вся страница' ---


### Разбор кода

Ключевой момент — и `Circle`, и `CompoundGraphic` наследуют `Graphic`. Это позволяет класть один `CompoundGraphic` внутрь другого, создавая **древовидную структуру объектов**:

```
all_graphics ("Вся страница")
│
├── Dot ("Точка 1")
├── Circle ("Круг 1")
└── group ("Группа фигур")
    │
    ├── Circle ("Внутренний круг")
    └── Dot ("Внутренняя точка")
```
Благодаря общему интерфейсу `Graphic`, клиентский код работает со всей этой сложной структурой так же просто, как с одним объектом.

## 2. Паттерн "Мост" (Bridge)

**Суть:** *Разделяет один или несколько классов на две отдельные иерархии (абстракцию и реализацию), что позволяет изменять их независимо друг от друга.*

**Аналогия:** Пульт и устройство. Есть разные пульты (простой, продвинутый) и разные устройства (ТВ, радио). Вместо того чтобы делать `ПродвинутыйПультДляТВ`, мы делаем `ПродвинутыйПульт`, который может работать с любым `Устройством`.

### Пример на Python

In [ ]:
from abc import ABC, abstractmethod

# Иерархия 2: Реализации (Устройства)
class Device(ABC):
    @abstractmethod
    def turn_on(self): pass
    @abstractmethod
    def turn_off(self): pass
    @abstractmethod
    def set_volume(self, percent: int): pass

class Tv(Device):
    def turn_on(self): print("Телевизор включен")
    def turn_off(self): print("Телевизор выключен")
    def set_volume(self, percent: int): print(f"Громкость ТВ: {percent}%")

class Radio(Device):
    def turn_on(self): print("Радио включено")
    def turn_off(self): print("Радио выключено")
    def set_volume(self, percent: int): print(f"Громкость радио: {percent}%")


# Иерархия 1: Абстракции (Пульты)
class RemoteControl:
    def __init__(self, device: Device):
        self._device = device  # <-- Вот он, МОСТ!

    def toggle_power(self):
        print("Нажата кнопка питания.")
        self._device.turn_on()

class AdvancedRemoteControl(RemoteControl):
    def mute(self):
        print("Нажата кнопка 'Mute'.")
        self._device.set_volume(0)

# Клиентский код
if __name__ == "__main__":
    tv = Tv()
    radio = Radio()

    print("--- Сценарий 1: Простой пульт с телевизором ---")
    remote_for_tv = RemoteControl(tv)
    remote_for_tv.toggle_power()
    print()

    print("--- Сценарий 2: Продвинутый пульт с радио ---")
    advanced_remote_for_radio = AdvancedRemoteControl(radio)
    advanced_remote_for_radio.toggle_power()
    advanced_remote_for_radio.mute()

--- Сценарий 1: Простой пульт с телевизором ---
Нажата кнопка питания.
Телевизор включен

--- Сценарий 2: Продвинутый пульт с радио ---
Нажата кнопка питания.
Радио включено
Нажата кнопка 'Mute'.
Громкость радио: 0%


### Разбор кода

Ключевая идея — **предпочитать композицию наследованию**. Вместо того чтобы `AdvancedTVRemote` наследовался от `TVRemote`, у нас есть две независимые ветки:
1.  **Пульты:** `RemoteControl` -> `AdvancedRemoteControl`.
2.  **Устройства:** `Device` -> `Tv`, `Radio`.

Класс `RemoteControl` **содержит** в себе объект `Device` и делегирует ему вызовы. Это позволяет нам независимо добавлять новые пульты и новые устройства, а затем свободно комбинировать их при создании объектов.

## 3. Паттерн "Легковес" (Flyweight)

**Суть:** *Позволяет вместить бóльшее количество объектов в отведённую оперативную память, разделяя общее состояние между ними.*

**Аналогия:** Буквы в текстовом редакторе. Чтобы отобразить текст из миллиона символов "а", не нужно создавать миллион объектов "буква 'а'". Достаточно создать один объект "буква 'а'" и затем миллион раз отрисовать его в разных координатах.

### Пример на Python

In [ ]:
import json

class TreeType:
    def __init__(self, name: str, color: str, texture: str):
        print(f"Создан новый, тяжелый объект TreeType: {name}, {color}")
        self._name = name
        self._color = color
        self._texture = texture # Представим, что это "тяжелые" данные

    def draw(self, x, y):
        print(f"Рисуем дерево '{self._name}' в ({x}, {y})")

class TreeFactory:
    _tree_types = {}
    @classmethod
    def get_tree_type(cls, name: str, color: str, texture: str) -> TreeType:
        key = (name, color, texture)
        if key not in cls._tree_types:
            cls._tree_types[key] = TreeType(name, color, texture)
        else:
            print(f"--> Используем существующий объект TreeType: {name}, {color}")

        return cls._tree_types[key]

class Tree:
    def __init__(self, x: int, y: int, tree_type: TreeType):
        self.x = x
        self.y = y
        self.tree_type = tree_type

    def draw(self):
        self.tree_type.draw(self.x, self.y)

class Forest:
    def __init__(self):
        self._trees: List[Tree] = []

    def plant_tree(self, x: int, y: int, name: str, color: str, texture: str):
        tree_type = TreeFactory.get_tree_type(name, color, texture)
        tree = Tree(x, y, tree_type)
        self._trees.append(tree)

    def draw(self):
        for tree in self._trees:
            tree.draw()

if __name__ == "__main__":
    forest = Forest()

    forest.plant_tree(10, 20, "Береза", "белый", "birch.png")
    forest.plant_tree(50, 80, "Ель", "зеленый", "spruce.png")
    forest.plant_tree(100, 150, "Береза", "белый", "birch.png")
    forest.plant_tree(200, 210, "Береза", "белый", "birch.png")

    print(f"\nВсего деревьев в лесу: {len(forest._trees)}")
    print(f"Всего создано тяжелых объектов (легковесов): {len(TreeFactory._tree_types)}\n")

    print("--- Рендеринг леса ---")
    forest.draw()

Создан новый, тяжелый объект TreeType: Береза, белый
Создан новый, тяжелый объект TreeType: Ель, зеленый
--> Используем существующий объект TreeType: Береза, белый
--> Используем существующий объект TreeType: Береза, белый

Всего деревьев в лесу: 4
Всего создано тяжелых объектов (легковесов): 2

--- Рендеринг леса ---
Рисуем дерево 'Береза' в (10, 20)
Рисуем дерево 'Ель' в (50, 80)
Рисуем дерево 'Береза' в (100, 150)
Рисуем дерево 'Береза' в (200, 210)


### Разбор кода

Несмотря на то, что мы посадили 4 дерева, в памяти было создано всего 2 "тяжелых" объекта `TreeType`. Остальные деревья просто хранят свои уникальные координаты (`x`, `y`) и ссылку на общий объект. Это позволяет экономить огромное количество памяти, когда речь идет о миллионах объектов.

## Заключение: Сравнительная таблица

| Паттерн | Цель | Ключевая идея | Структура |
| :--- | :--- | :--- | :--- |
| **Компоновщик** | Построить древовидную иерархию и работать с ней как с единым целым. | **Единообразие.** Клиент не различает простые и составные объекты. | Общий интерфейс для листьев и контейнеров. |
| **Мост** | Разделить большую иерархию классов на две независимые, чтобы избежать "комбинаторного взрыва". | **Разделение.** Абстракция отделена от реализации. | Две отдельные иерархии, связанные через композицию. |
| **Легковес** | Эффективно использовать память при работе с большим числом однотипных объектов. | **Разделение общего состояния.** Объекты переиспользуются. | Фабрика, управляющая пулом объектов с общим состоянием. |

### Спасибо за внимание!